In [178]:
from iquhack_scoring import MoveScorer
from bloqade import move
from matplotlib.animation import FuncAnimation, PillowWriter
from numpy import pi
from helper_functions import *
from answers import *
import numpy as np
import cirq
from cirq.contrib.qasm_import import circuit_from_qasm
import re
import copy
from math import floor

In [5]:
f_name = "QAOA_qasm_str.txt"
with open(f_name, "r") as f:
    prog = f.read()
# print(prog)

In [6]:
circ = circuit_from_qasm(prog)

In [145]:
def storage_to_gate(start:list[int],end:list[int]):
    @move.vmove()
    def kernel(state:move.core.AtomState):
        state.gate[end] = move.Move(state.storage[start])
        return state
    return kernel

def gate_to_storage(start:list[int],end:list[int]):
    @move.vmove()
    def kernel(state:move.core.AtomState):
        state.storage[end] = move.Move(state.gate[start])
        return state
    return kernel

def do_H(indices:list[int]):
    @move.vmove()
    def kernel(state:move.core.AtomState):
        state = local_H(atom_state=state,indices=indices)
        return state
    return kernel

def do_CX(target_indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = local_CX(atom_state=state,target_indices=target_indices)
        return state
    return kernel

def do_U3(a:float, b:float, c:float, indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalRz(atom_state=state, phi=a, indices=indices)
        state = move.LocalXY(atom_state=state, x_exponent=b,axis_phase_exponent=0.5, indices=indices)
        state = move.LocalRz(atom_state=state, phi=c, indices=indices)
        return state
    return kernel

def do_CZ(target_indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.GlobalCZ(atom_state=state)
        return state
    return kernel

def do_X(indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalXY(atom_state=state,x_exponent=pi,axis_phase_exponent=0.0,indices=indices)
        return state
    return kernel

def do_Y(indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalXY(atom_state=state,x_exponent=pi,axis_phase_exponent=pi/2,indices=indices)
        return state
    return kernel

def do_Z(indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalRz(atom_state=state,phi=pi,indices=indices)
        return state
    return kernel

def do_Rx(angle:float, indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalXY(atom_state=state,x_exponent=angle,axis_phase_exponent=0.0,indices=indices)
        return state
    return kernel

def do_Ry(angle:float, indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalXY(atom_state=state,x_exponent=angle,axis_phase_exponent=pi/2,indices=indices)
        return state
    return kernel

def do_Rz(angle:float, indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalRz(atom_state=state,phi=angle,indices=indices)
        return state
    return kernel

def do_S(indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalRz(atom_state=state,phi=pi/2,indices=indices)
        return state
    return kernel

def do_T(indices:list[int]):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalRz(atom_state=state,phi=pi/4,indices=indices)
        return state
    return kernel

def do_sqrtX(indices:list[int], dag):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalXY(atom_state=state,x_exponent=pi/2 * (-1)**dag,axis_phase_exponent=0.0,indices=indices)
        return state
    return kernel

def do_sqrtY(indices:list[int], dag):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = move.LocalXY(atom_state=state,x_exponent=pi/2 * (-1)**dag,axis_phase_exponent=pi/2,indices=indices)
        return state
    return kernel

In [166]:
def join_ops(op1, op2):
    @move.vmove
    def kernel(state:move.core.AtomState):
        state = op1(state)
        state = op2(state)
        return state
    return kernel

In [195]:
n_qubits = len(circ.moments[0].qubits)

In [159]:
do_print = True

all_moment_ops = []
all_op_qubits = []
for i, moment in enumerate(circ.moments):
    # names of operators
    moment_ops = []
    # numbers of qubits corresponding to moment_ops
    op_qubits = [[] for i in range(n_qubits)]
    # op_qubits = np.zeros((len(moment.operations), 0))
    # loops through all operators
    for j, op1 in enumerate(moment.operations):
        # if operator is not in moment_ops, add it and add self to op_qubits, else, add self to op_qubits
        if str(op1.gate) not in moment_ops:
            moment_ops.append(str(op1.gate))
        for k, qubit in enumerate(op1.qubits):
            op_qubits[moment_ops.index(str(op1.gate))].append(int(qubit.name[2:]))

    for j in range(len(moment_ops)):
        all_moment_ops.append(moment_ops[j])
        if moment_ops[j] == "CNOT":
            all_op_qubits.append(op_qubits[j])
        else:
            op_qubits[j].sort()
            all_op_qubits.append(op_qubits[j])

if do_print:
    for i in range(len(all_moment_ops)):
        print(f'{i}: op: {all_moment_ops[i]}, qubits: {all_op_qubits[i]}')

0: op: H, qubits: [0, 1, 2, 3]
1: op: cirq.circuits.qasm_output.QasmUGate(theta=0.5, phi=0.0, lmda=0.25), qubits: [0]
2: op: cirq.circuits.qasm_output.QasmUGate(theta=0.5, phi=0.0, lmda=0.75), qubits: [1, 2, 3]
3: op: X**0.5, qubits: [0]
4: op: CNOT, qubits: [0, 1]
5: op: Rx(0.4223785852π), qubits: [0]
6: op: Ry(0.5π), qubits: [1]
7: op: CNOT, qubits: [1, 0]
8: op: X**-0.5, qubits: [1]
9: op: S, qubits: [1]
10: op: CNOT, qubits: [0, 1]
11: op: cirq.circuits.qasm_output.QasmUGate(theta=0.5, phi=0.8276214148, lmda=1.0), qubits: [0]
12: op: cirq.circuits.qasm_output.QasmUGate(theta=0.5, phi=0.3276214148, lmda=1.0), qubits: [1]
13: op: cirq.circuits.qasm_output.QasmUGate(theta=0.5, phi=0.0, lmda=0.25), qubits: [0, 1]
14: op: X**0.5, qubits: [0, 1]
15: op: CNOT, qubits: [0, 3]
16: op: Rx(0.4223785852π), qubits: [0]
17: op: Ry(0.5π), qubits: [3]
18: op: CNOT, qubits: [3, 0]
19: op: X**-0.5, qubits: [3]
20: op: S, qubits: [3]
21: op: CNOT, qubits: [0, 3]
22: op: cirq.circuits.qasm_output.Qasm

In [194]:
# This is the worst code I've ever written and I feel like throwing up just looking at it

n_ops = len(all_moment_ops)
na_ops = []
for i, op in enumerate(all_moment_ops):

    # move qubits to gate area
    if op == "CNOT":
        qbs = copy.deepcopy(all_op_qubits[i])
        qbs.sort()
        na_ops.append(storage_to_gate([0, 1], qbs))
    else:
        na_ops.append(storage_to_gate(all_op_qubits[i], all_op_qubits[i]))

    # do gate
    if op == "H":
        na_ops.append(do_H(indices=all_op_qubits[i]))
    elif "QasmUGate" in op:
        theta, phi, lmda = re.findall(r"[-+]?(?:\d*\.*\d+)", op)
        na_ops.append(do_U3(theta, phi, lmda, indices=all_op_qubits[i]))
    elif op == "CNOT":
        if all_op_qubits[i][0] < all_op_qubits[i][0]:
            target = 1
        else:
            target = 0
        na_ops.append(do_CX(target_indices=target))
    elif op == "X**0.5":
        na_ops.append(do_sqrtX(indices=all_op_qubits[i],dag=False))
    elif op == "X**-0.5":
        na_ops.append(do_sqrtX(indices=all_op_qubits[i],dag=True))
    elif op == "Y**0.5":
        na_ops.append(do_sqrtY(indices=all_op_qubits[i],dag=False))
    elif op == "Y**-0.5":
        na_ops.append(do_sqrtY(indices=all_op_qubits[i],dag=True))
    elif op == "S":
        na_ops.append(do_S(indices=all_op_qubits[i]))
    elif op == "T":
        na_ops.append(do_T(indices=all_op_qubits[i]))
    elif op[:2] == "Rx":
        angle = re.findall(r"[-+]?(?:\d*\.*\d+)", op)[0]
        na_ops.append(do_Rx(angle,indices=all_op_qubits[i]))
    elif op[:2] == "Ry":
        angle = re.findall(r"[-+]?(?:\d*\.*\d+)", op)[0]
        na_ops.append(do_Ry(angle,indices=all_op_qubits[i]))
    elif op[:2] == "Rz":
        angle = re.findall(r"[-+]?(?:\d*\.*\d+)", op)[0]
        na_ops.append(do_Rz(angle,indices=all_op_qubits[i]))

    # move qubits back
    if op == "CNOT":
        qbs = copy.deepcopy(all_op_qubits[i])
        qbs.sort()
        na_ops.append(gate_to_storage(qbs, [0, 1]))
    else:
        na_ops.append(gate_to_storage(all_op_qubits[i], all_op_qubits[i]))

temp_op_arr = []
for i in range(0, floor(len(na_ops) / 2), 2):
    temp_op_arr.append(join_ops(na_ops[2 * i], na_ops[(2 * i)+1]))
if len(na_ops) % 2 == 1:
    temp_op_arr.append(na_ops[-1])

temp_op_arr2 = []
for i in range(floor(len(temp_op_arr) / 2)):
    temp_op_arr2.append(join_ops(temp_op_arr[2 * i], temp_op_arr[(2 * i)+1]))
if len(temp_op_arr) % 2 == 1:
    temp_op_arr2.append(temp_op_arr[-1])

temp_op_arr3 = []
for i in range(floor(len(temp_op_arr2) / 2)):
    temp_op_arr3.append(join_ops(temp_op_arr2[2 * i], temp_op_arr2[(2 * i)+1]))
if len(temp_op_arr2) % 2 == 1:
    temp_op_arr3.append(temp_op_arr2[-1])

temp_op_arr4 = []
for i in range(floor(len(temp_op_arr3) / 2)):
    temp_op_arr4.append(join_ops(temp_op_arr3[2 * i], temp_op_arr3[(2 * i)+1]))
if len(temp_op_arr3) % 2 == 1:
    temp_op_arr4.append(temp_op_arr3[-1])

temp_op_arr5 = []
for i in range(floor(len(temp_op_arr4) / 2)):
    temp_op_arr5.append(join_ops(temp_op_arr4[2 * i], temp_op_arr4[(2 * i)+1]))
if len(temp_op_arr4) % 2 == 1:
    temp_op_arr5.append(temp_op_arr4[-1])

temp_op_arr6 = []
for i in range(floor(len(temp_op_arr5) / 2)):
    temp_op_arr6.append(join_ops(temp_op_arr5[2 * i], temp_op_arr5[(2 * i)+1]))
if len(temp_op_arr5) % 2 == 1:
    temp_op_arr6.append(temp_op_arr5[-1])

temp_op_arr7 = []
for i in range(floor(len(temp_op_arr6) / 2)):
    temp_op_arr7.append(join_ops(temp_op_arr6[2 * i], temp_op_arr6[(2 * i)+1]))
if len(temp_op_arr6) % 2 == 1:
    temp_op_arr7.append(temp_op_arr6[-1])

temp_op_arr8 = []
for i in range(floor(len(temp_op_arr7) / 2)):
    temp_op_arr8.append(join_ops(temp_op_arr7[2 * i], temp_op_arr7[(2 * i)+1]))
if len(temp_op_arr7) % 2 == 1:
    temp_op_arr8.append(temp_op_arr8[-1])


print(len(temp_op_arr8))
print(temp_op_arr8)

1
[Method("kernel")]


In [165]:
print(n_ops)
print(len(na_ops))

140
420
